In [1]:
# ============================================
# Cell 0: Load Model for Deployment
# ============================================

import torch
import torch.nn as nn
import json
import os
from torchvision.models import efficientnet_b0
from torchvision import transforms
from PIL import Image
import numpy as np

# Load config
with open("model_config.json", "r") as f:
    config = json.load(f)

print("📋 Model Config:")
for k, v in config.items():
    if not isinstance(v, dict):
        print(f"   {k}: {v}")

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = efficientnet_b0(weights=None)
model.classifier[1] = nn.Linear(1280, config["num_classes"])
model.load_state_dict(torch.load("best_model_crop.pt", map_location=device))
model = model.to(device)
model.eval()

print(f"\n✅ Model loaded on {device}")
print(f"✅ Config loaded — ready for deployment")

# Center crop transform (same as training)
class DynamicCenterCrop:
    def __init__(self, ratio=0.7):
        self.ratio = ratio
    def __call__(self, img):
        w, h = img.size
        new_w = int(w * self.ratio)
        new_h = int(h * self.ratio)
        left = (w - new_w) // 2
        top = (h - new_h) // 2
        return img.crop((left, top, left + new_w, top + new_h))

inference_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    DynamicCenterCrop(config["center_crop_ratio"]),
    transforms.Resize((config["input_size"], config["input_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=config["normalization"]["mean"], 
                         std=config["normalization"]["std"])
])

# Quick test
print("\n🧪 Quick inference test...")
test_img = Image.new('RGB', (300, 300), color='gray')
tensor = inference_transform(test_img).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(tensor)
    probs = torch.softmax(output, dim=1)
print(f"   Test prediction: {config['class_names'][probs.argmax().item()]} ({probs.max().item()*100:.1f}%)")
print("✅ Inference pipeline working!")

📋 Model Config:
   model_name: EfficientNet-B0
   input_size: 224
   num_classes: 2
   class_names: ['NORMAL', 'PNEUMONIA']
   center_crop_ratio: 0.7

✅ Model loaded on cuda
✅ Config loaded — ready for deployment

🧪 Quick inference test...
   Test prediction: PNEUMONIA (100.0%)
✅ Inference pipeline working!


In [5]:
# ============================================
# Cell 1: Generate Deployment Files
# ============================================

print("=" * 60)
print("  GENERATING DEPLOYMENT FILES")
print("=" * 60)

# ============================================
# FILE 1: FastAPI Backend (app.py)
# ============================================

fastapi_code = '''
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0
from torchvision import transforms
from PIL import Image
import numpy as np
import io
import json
import base64

# ---- App Setup ----
app = FastAPI(
    title="Pneumonia Detection API",
    description="API de détection de pneumonie par analyse d'images radiographiques (Rayons X)",
    version="1.0.0",
    contact={"name": "Ahmed Ben Attia Khiari & Achref Ghorbel"}
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# ---- Model Loading ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("model_config.json", "r") as f:
    config = json.load(f)

model = efficientnet_b0(weights=None)
model.classifier[1] = nn.Linear(1280, config["num_classes"])
model.load_state_dict(torch.load("best_model_crop.pt", map_location=device))
model = model.to(device)
model.eval()

# ---- Transforms ----
class DynamicCenterCrop:
    def __init__(self, ratio=0.7):
        self.ratio = ratio
    def __call__(self, img):
        w, h = img.size
        new_w = int(w * self.ratio)
        new_h = int(h * self.ratio)
        left = (w - new_w) // 2
        top = (h - new_h) // 2
        return img.crop((left, top, left + new_w, top + new_h))

inference_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    DynamicCenterCrop(config["center_crop_ratio"]),
    transforms.Resize((config["input_size"], config["input_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=config["normalization"]["mean"],
                         std=config["normalization"]["std"])
])

raw_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    DynamicCenterCrop(config["center_crop_ratio"]),
    transforms.Resize((config["input_size"], config["input_size"])),
])

# ---- Grad-CAM ----
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = None
        self.activations = None
        target_layer.register_forward_hook(self._fwd)
        target_layer.register_full_backward_hook(self._bwd)

    def _fwd(self, module, input, output):
        self.activations = output.detach()

    def _bwd(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def generate(self, input_tensor, target_class=None):
        self.model.eval()
        output = self.model(input_tensor)
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        self.model.zero_grad()
        output[0, target_class].backward()
        weights = self.gradients.mean(dim=[2, 3], keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = torch.relu(cam)
        cam = torch.nn.functional.interpolate(cam, size=(224, 224), mode="bilinear", align_corners=False)
        cam = cam.squeeze().cpu().numpy()
        if cam.max() > 0:
            cam = cam / cam.max()
        return cam, output

grad_cam = GradCAM(model, model.features[-1])

def generate_gradcam_image(img: Image.Image) -> str:
    """Generate Grad-CAM overlay and return as base64 string."""
    import matplotlib
    matplotlib.use("Agg")
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    tensor = inference_transform(img).unsqueeze(0).to(device)
    heatmap, _ = grad_cam.generate(tensor)
    
    raw_img = np.array(raw_transform(img))
    
    colormap = cm.jet(heatmap)[:, :, :3]
    colormap = (colormap * 255).astype(np.uint8)
    
    if raw_img.ndim == 2:
        raw_img = np.stack([raw_img]*3, axis=-1)
    
    overlay = (0.6 * raw_img + 0.4 * colormap).astype(np.uint8)
    
    overlay_img = Image.fromarray(overlay)
    buffer = io.BytesIO()
    overlay_img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# ---- Routes ----
@app.get("/")
async def root():
    return {
        "message": "Pneumonia Detection API",
        "version": "1.0.0",
        "endpoints": {
            "/predict": "POST - Upload X-ray image for prediction",
            "/metrics": "GET - Model performance metrics",
            "/health": "GET - API health check"
        }
    }

@app.get("/health")
async def health():
    return {"status": "healthy", "device": str(device), "model": config["model_name"]}

@app.get("/metrics")
async def metrics():
    return {
        "model": config["model_name"],
        "metrics": config["metrics"],
        "training": config["training"]
    }

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    # Validate file
    if not file.content_type or not file.content_type.startswith("image/"):
        raise HTTPException(status_code=400, detail="Le fichier doit être une image (JPEG, PNG)")
    
    try:
        contents = await file.read()
        img = Image.open(io.BytesIO(contents))
    except Exception:
        raise HTTPException(status_code=400, detail="Impossible de lire l'image")
    
    # Predict
    tensor = inference_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(tensor)
        probs = torch.softmax(output, dim=1)
    
    pred_idx = probs.argmax(dim=1).item()
    confidence = probs[0][pred_idx].item()
    
    # Grad-CAM
    gradcam_b64 = generate_gradcam_image(img)
    
    return JSONResponse(content={
        "prediction": config["class_names"][pred_idx],
        "confidence": round(confidence * 100, 2),
        "probabilities": {
            "NORMAL": round(probs[0][0].item() * 100, 2),
            "PNEUMONIA": round(probs[0][1].item() * 100, 2)
        },
        "gradcam_image": gradcam_b64
    })

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open("app.py", "w") as f:
    f.write(fastapi_code.strip())
print("✅ Created: app.py (FastAPI backend)")

# ============================================
# FILE 2: Streamlit Frontend (streamlit_app.py)
# ============================================

streamlit_code = '''
import streamlit as st
import requests
from PIL import Image
import io
import base64
import json

# ---- Config ----
API_URL = "http://localhost:8000"

# ---- Page Setup ----
st.set_page_config(
    page_title="Détection de Pneumonie",
    page_icon="🫁",
    layout="wide"
)

# ---- Custom CSS ----
st.markdown("""
<style>
    .main-header {
        text-align: center;
        padding: 1rem;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 10px;
        margin-bottom: 2rem;
    }
    .metric-card {
        background: #f8f9fa;
        padding: 1.5rem;
        border-radius: 10px;
        border-left: 4px solid;
        margin: 0.5rem 0;
    }
    .normal-result {
        background: #d4edda;
        border-color: #28a745;
        padding: 2rem;
        border-radius: 10px;
        text-align: center;
    }
    .pneumonia-result {
        background: #f8d7da;
        border-color: #dc3545;
        padding: 2rem;
        border-radius: 10px;
        text-align: center;
    }
</style>
""", unsafe_allow_html=True)

# ---- Header ----
st.markdown("""
<div class="main-header">
    <h1>🫁 Détection de Pneumonie par Rayons X</h1>
    <p>Système d'aide au diagnostic basé sur le Deep Learning (EfficientNet-B0)</p>
    <p><em>Ahmed Ben Attia Khiari & Achref Ghorbel — Polytech International</em></p>
</div>
""", unsafe_allow_html=True)

# ---- Sidebar ----
with st.sidebar:
    st.header("ℹ️ À propos")
    st.markdown("""
    Ce système utilise un modèle **EfficientNet-B0** entraîné sur le dataset 
    **Chest X-Ray Images** pour détecter la pneumonie dans les radiographies thoraciques.
    
    **⚠️ Avertissement:** Ce système est un outil d'aide au diagnostic à usage académique 
    uniquement. Il ne remplace pas l'avis d'un professionnel de santé.
    """)
    
    st.header("📊 Métriques du Modèle")
    try:
        resp = requests.get(f"{API_URL}/metrics", timeout=5)
        if resp.status_code == 200:
            data = resp.json()
            m = data["metrics"]
            st.metric("Accuracy", f"{m['accuracy']*100:.1f}%")
            st.metric("AUC-ROC", f"{m['auc_roc']:.4f}")
            st.metric("Recall", f"{m['recall']*100:.1f}%")
            st.metric("Specificity", f"{m['specificity']*100:.1f}%")
            st.metric("F1-Score", f"{m['f1_score']:.4f}")
        else:
            st.warning("API non disponible")
    except:
        st.warning("⚠️ API non connectée. Lancez: `uvicorn app:app --port 8000`")

# ---- Main Content ----
col1, col2 = st.columns(2)

with col1:
    st.header("📤 Upload Radiographie")
    uploaded_file = st.file_uploader(
        "Choisir une image radiographique (JPEG, PNG)",
        type=["jpg", "jpeg", "png"],
        help="Uploadez une radiographie thoracique pour analyse"
    )
    
    if uploaded_file:
        image = Image.open(uploaded_file)
        st.image(image, caption="Image uploadée", use_container_width=True)

with col2:
    st.header("🔍 Résultat")
    
    if uploaded_file:
        with st.spinner("Analyse en cours..."):
            try:
                files = {"file": (uploaded_file.name, uploaded_file.getvalue(), uploaded_file.type)}
                response = requests.post(f"{API_URL}/predict", files=files, timeout=30)
                
                if response.status_code == 200:
                    result = response.json()
                    prediction = result["prediction"]
                    confidence = result["confidence"]
                    
                    # Result display
                    if prediction == "NORMAL":
                        st.markdown(f"""
                        <div class="normal-result">
                            <h2>✅ {prediction}</h2>
                            <h3>Confiance: {confidence}%</h3>
                            <p>Aucun signe de pneumonie détecté</p>
                        </div>
                        """, unsafe_allow_html=True)
                    else:
                        st.markdown(f"""
                        <div class="pneumonia-result">
                            <h2>⚠️ {prediction}</h2>
                            <h3>Confiance: {confidence}%</h3>
                            <p>Signes de pneumonie détectés — Consultez un médecin</p>
                        </div>
                        """, unsafe_allow_html=True)
                    
                    # Probabilities
                    st.subheader("📊 Probabilités")
                    probs = result["probabilities"]
                    col_a, col_b = st.columns(2)
                    col_a.metric("Normal", f"{probs['NORMAL']}%")
                    col_b.metric("Pneumonie", f"{probs['PNEUMONIA']}%")
                    
                    # Progress bar
                    st.progress(probs["PNEUMONIA"] / 100)
                    
                    # Grad-CAM
                    st.subheader("🔥 Carte d'attention (Grad-CAM)")
                    st.caption("Les zones rouges/jaunes indiquent où le modèle concentre son attention")
                    gradcam_bytes = base64.b64decode(result["gradcam_image"])
                    gradcam_img = Image.open(io.BytesIO(gradcam_bytes))
                    st.image(gradcam_img, caption="Grad-CAM Overlay", use_container_width=True)
                    
                else:
                    st.error(f"Erreur API: {response.status_code}")
                    
            except requests.exceptions.ConnectionError:
                st.error("❌ Impossible de se connecter à l'API. Lancez: `uvicorn app:app --port 8000`")
            except Exception as e:
                st.error(f"Erreur: {str(e)}")
    else:
        st.info("👈 Uploadez une radiographie pour commencer l'analyse")

# ---- Footer ----
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: gray;">
    <p>Projet Deep Learning — Détection de Pneumonie | Polytech International 2026</p>
    <p>Professeur: Haythem Ghazouani</p>
</div>
""", unsafe_allow_html=True)
'''

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code.strip())
print("✅ Created: streamlit_app.py (Streamlit frontend)")

# ============================================
# FILE 3: Dockerfile
# ============================================

dockerfile = '''FROM python:3.10-slim

WORKDIR /app

# Install system deps
RUN apt-get update && apt-get install -y --no-install-recommends \\
    libgl1-mesa-glx libglib2.0-0 && \\
    rm -rf /var/lib/apt/lists/*

# Install Python deps
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy app files
COPY app.py .
COPY streamlit_app.py .
COPY best_model_crop.pt .
COPY model_config.json .

EXPOSE 8000 8501
'''

with open("Dockerfile", "w") as f:
    f.write(dockerfile.strip())
print("✅ Created: Dockerfile")

# ============================================
# FILE 4: docker-compose.yml
# ============================================

compose = '''version: "3.8"

services:
  api:
    build: .
    container_name: pneumonia-api
    command: uvicorn app:app --host 0.0.0.0 --port 8000
    ports:
      - "8000:8000"
    volumes:
      - ./best_model_crop.pt:/app/best_model_crop.pt
      - ./model_config.json:/app/model_config.json

  frontend:
    build: .
    container_name: pneumonia-frontend
    command: streamlit run streamlit_app.py --server.port 8501 --server.address 0.0.0.0
    ports:
      - "8501:8501"
    depends_on:
      - api
    environment:
      - API_URL=http://api:8000

  mlflow:
    image: python:3.10-slim
    container_name: pneumonia-mlflow
    command: bash -c "pip install mlflow && mlflow ui --host 0.0.0.0 --port 5000"
    ports:
      - "5000:5000"
    volumes:
      - ./mlruns:/mlruns
'''

with open("docker-compose.yml", "w") as f:
    f.write(compose.strip())
print("✅ Created: docker-compose.yml")

# ============================================
# FILE 5: requirements.txt
# ============================================

requirements = '''torch>=2.0.0
torchvision>=0.15.0
fastapi>=0.100.0
uvicorn>=0.22.0
python-multipart>=0.0.6
streamlit>=1.30.0
Pillow>=9.0.0
numpy>=1.24.0
matplotlib>=3.7.0
requests>=2.28.0
mlflow>=2.10.0
'''

with open("requirements.txt", "w") as f:
    f.write(requirements.strip())
print("✅ Created: requirements.txt")

# ============================================
# FILE 6: test_app.py (Pytest)
# ============================================

test_code = '''
import pytest
from fastapi.testclient import TestClient
from app import app
from PIL import Image
import io
import json
import os

client = TestClient(app)

# ---- Test Health ----
def test_health():
    response = client.get("/health")
    assert response.status_code == 200
    data = response.json()
    assert data["status"] == "healthy"
    assert "device" in data
    assert data["model"] == "EfficientNet-B0"

# ---- Test Root ----
def test_root():
    response = client.get("/")
    assert response.status_code == 200
    data = response.json()
    assert "endpoints" in data

# ---- Test Metrics ----
def test_metrics():
    response = client.get("/metrics")
    assert response.status_code == 200
    data = response.json()
    assert "metrics" in data
    assert data["metrics"]["accuracy"] > 0.90
    assert data["metrics"]["auc_roc"] > 0.90

# ---- Test Predict with valid image ----
def test_predict_valid_image():
    img = Image.new("RGB", (224, 224), color="gray")
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    buffer.seek(0)
    
    response = client.post("/predict", files={"file": ("test.jpg", buffer, "image/jpeg")})
    assert response.status_code == 200
    data = response.json()
    assert "prediction" in data
    assert data["prediction"] in ["NORMAL", "PNEUMONIA"]
    assert "confidence" in data
    assert 0 <= data["confidence"] <= 100
    assert "probabilities" in data
    assert "gradcam_image" in data

# ---- Test Predict with different sizes ----
def test_predict_different_sizes():
    for size in [(100, 100), (500, 500), (1024, 768)]:
        img = Image.new("RGB", size, color="white")
        buffer = io.BytesIO()
        img.save(buffer, format="JPEG")
        buffer.seek(0)
        response = client.post("/predict", files={"file": ("test.jpg", buffer, "image/jpeg")})
        assert response.status_code == 200

# ---- Test Predict with PNG ----
def test_predict_png():
    img = Image.new("RGB", (224, 224), color="gray")
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    buffer.seek(0)
    response = client.post("/predict", files={"file": ("test.png", buffer, "image/png")})
    assert response.status_code == 200

# ---- Test Config Exists ----
def test_config_exists():
    assert os.path.exists("model_config.json")
    with open("model_config.json", "r") as f:
        config = json.load(f)
    assert config["model_name"] == "EfficientNet-B0"
    assert config["num_classes"] == 2
    assert config["center_crop_ratio"] == 0.7

# ---- Test Model File Exists ----
def test_model_exists():
    assert os.path.exists("best_model_crop.pt")

# ---- Test Gradcam is valid base64 ----
def test_gradcam_base64():
    img = Image.new("RGB", (224, 224), color="gray")
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    buffer.seek(0)
    response = client.post("/predict", files={"file": ("test.jpg", buffer, "image/jpeg")})
    data = response.json()
    import base64
    decoded = base64.b64decode(data["gradcam_image"])
    assert len(decoded) > 0
    gradcam_img = Image.open(io.BytesIO(decoded))
    assert gradcam_img.size[0] > 0
'''

with open("test_app.py", "w") as f:
    f.write(test_code.strip())
print("✅ Created: test_app.py (Pytest tests)")

# ============================================
# Summary
# ============================================
print(f"\n{'='*60}")
print(f"  ALL DEPLOYMENT FILES CREATED")
print(f"{'='*60}")
print(f"""
📁 Project Structure:
├── best_model_crop.pt      (trained model)
├── model_config.json       (config & metrics)
├── app.py                  (FastAPI backend)
├── streamlit_app.py        (Streamlit frontend)
├── test_app.py             (Pytest tests)
├── requirements.txt        (dependencies)
├── Dockerfile              
├── docker-compose.yml      
└── projetFederaturBDBI.ipynb (training notebook)

🚀 TO RUN LOCALLY:
   1. pip install -r requirements.txt
   2. uvicorn app:app --reload --port 8000
   3. streamlit run streamlit_app.py (new terminal)

🐳 TO RUN WITH DOCKER:
   docker-compose up --build

🧪 TO RUN TESTS:
   pytest test_app.py -v
""")

  GENERATING DEPLOYMENT FILES
✅ Created: app.py (FastAPI backend)
✅ Created: streamlit_app.py (Streamlit frontend)
✅ Created: Dockerfile
✅ Created: docker-compose.yml
✅ Created: requirements.txt
✅ Created: test_app.py (Pytest tests)

  ALL DEPLOYMENT FILES CREATED

📁 Project Structure:
├── best_model_crop.pt      (trained model)
├── model_config.json       (config & metrics)
├── app.py                  (FastAPI backend)
├── streamlit_app.py        (Streamlit frontend)
├── test_app.py             (Pytest tests)
├── requirements.txt        (dependencies)
├── Dockerfile              
├── docker-compose.yml      
└── projetFederaturBDBI.ipynb (training notebook)

🚀 TO RUN LOCALLY:
   1. pip install -r requirements.txt
   2. uvicorn app:app --reload --port 8000
   3. streamlit run streamlit_app.py (new terminal)

🐳 TO RUN WITH DOCKER:
   docker-compose up --build

🧪 TO RUN TESTS:
   pytest test_app.py -v



In [13]:
# ============================================
# Cell 2: Generate README
# ============================================

readme = r'''# 🫁 Détection de Pneumonie par Analyse d'Images Radiographiques

Système d'aide au diagnostic basé sur le Deep Learning capable de classifier automatiquement les images radiographiques thoraciques (Rayons X) en deux catégories : **Normal** ou **Pneumonie**.

## 👥 Équipe
- **Ahmed Ben Attia Khiari**
- **Achref Ghorbel**

**Professeur :** Haythem Ghazouani  
**Module :** Deep Learning — Computer Vision  
**Institution :** Polytech International, 2026

## 📊 Résultats

| Métrique | Objectif | Résultat |
|----------|----------|----------|
| Accuracy | > 90% | **97.9%** ✅ |
| AUC-ROC | > 0.90 | **0.9976** ✅ |
| Recall | > 85% | **98.6%** ✅ |
| Specificity | > 85% | **96.2%** ✅ |
| F1-Score | > 0.88 | **0.9860** ✅ |

## 🏗️ Architecture
```
Frontend (Streamlit:8501) → Backend (FastAPI:8000) → Modèle (EfficientNet-B0)
                                                    ↓
                                              MLflow (5000)
```

### Stack Technique
- **Modèle :** EfficientNet-B0 (pré-entraîné ImageNet, fine-tuné)
- **Framework :** PyTorch
- **Backend :** FastAPI (API REST avec documentation Swagger)
- **Frontend :** Streamlit
- **Tracking :** MLflow
- **Tests :** Pytest (9/9 ✅)
- **Conteneurisation :** Docker + Docker Compose

## 📁 Structure du Projet
```
├── projetFederaturBDBI.ipynb   # Notebook: EDA + Training + Evaluation
├── deployment.ipynb            # Notebook: Déploiement
├── app.py                      # Backend FastAPI
├── streamlit_app.py            # Frontend Streamlit
├── test_app.py                 # Tests Pytest
├── best_model_crop.pt          # Modèle entraîné
├── model_config.json           # Configuration du modèle
├── requirements.txt            # Dépendances Python
├── Dockerfile                  # Image Docker
├── docker-compose.yml          # Orchestration Docker
├── Cahier_de_Charges.docx      # Cahier de charges
└── chest_xray/                 # Dataset
    ├── train/
    │   ├── NORMAL/
    │   └── PNEUMONIA/
    └── test/
        ├── NORMAL/
        └── PNEUMONIA/
```

## 🔬 Méthodologie

### 1. Analyse Exploratoire (EDA)
- Dataset : 5,840 images (Chest X-Ray Images — Kaggle)
- Distribution : 1,575 NORMAL / 4,265 PNEUMONIA (déséquilibré)
- Source : Guangzhou Women and Children's Medical Center

### 2. Prétraitement & Augmentation
- Conversion RGB, redimensionnement 224×224
- Center Crop (70%) pour éliminer les artefacts de bord (IVs, tubes)
- Augmentation : RandomHorizontalFlip, RandomRotation(10°), ColorJitter, RandomAffine
- Normalisation ImageNet : mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]

### 3. Entraînement
- Architecture : EfficientNet-B0 (4M paramètres)
- Optimiseur : Adam (LR=0.0005)
- Scheduler : ReduceLROnPlateau (patience=2)
- Split : 70% train / 15% val / 15% test (seed=42)
- 12 époques, meilleure validation : 97.6%

### 4. Étude d'Ablation

| Configuration | Accuracy | Specificity | Recall |
|---------------|----------|-------------|--------|
| V1: Class weights lourds | 85.6% | 62.0% | 99.7% |
| V2: Class weights sqrt | 89.9% | 74.8% | 99.0% |
| V4: Split 70/15/15 | 97.9% | 95.3% | 98.9% |
| V5: + Center Crop (FINAL) | 97.9% | 96.2% | 98.6% |

### 5. Interprétabilité (Grad-CAM)
- Détection de **shortcut learning** : le modèle initial détectait les IVs/tubes au lieu des pathologies pulmonaires
- Correction par Center Crop (70%) : ratio d'attention Pneumonia/Normal amélioré de 0.36 → 0.68
- Le modèle final se concentre correctement sur les patterns pulmonaires

## 🚀 Installation & Utilisation

### Prérequis
- Python 3.10+
- CUDA compatible GPU (recommandé)

### Installation
```bash
pip install -r requirements.txt
```

### Exécution locale
```bash
# Terminal 1: API Backend
uvicorn app:app --port 8000

# Terminal 2: Frontend
streamlit run streamlit_app.py
```
Ouvrir http://localhost:8501 dans le navigateur.

### Docker
```bash
docker-compose up --build
```
- Frontend : http://localhost:8501
- API : http://localhost:8000
- API Docs : http://localhost:8000/docs
- MLflow : http://localhost:5000

### Tests
```bash
pytest test_app.py -v
```

## ⚠️ Considérations Éthiques
- Ce système est un outil d'aide au diagnostic à **usage académique uniquement**
- Il ne remplace pas l'avis d'un professionnel de santé
- Le dataset provient d'un seul hôpital (enfants chinois) — biais potentiel
- Validation clinique nécessaire avant tout usage médical réel

## 📈 MLflow
Les expériences sont trackées avec MLflow. Pour visualiser :
```bash
mlflow ui --port 5000
```
'''

# Save
target = r"C:\Users\mbena\Downloads\datasetProjetfed"

with open(os.path.join(target, "README.md"), "w", encoding="utf-8") as f:
    f.write(readme.strip())

print("✅ README.md created!")
print(f"   Location: {os.path.join(target, 'README.md')}")

✅ README.md created!
   Location: C:\Users\mbena\Downloads\datasetProjetfed\README.md
